In [ ]:
import speech_recognition as srec
from gtts import gTTS
import pyttsx3 as pyt
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, VitsModel
import torch

import time
import psutil

import whisper
import sounddevice as sd
import numpy as np
from IPython.display import Audio

In [ ]:
engine = pyt.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

device = "auto"

In [ ]:
# --- LLAMA ---

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [ ]:
# --- STT ---

# def perintah():
#     mendengar = srec.Recognizer()
#     with srec.Microphone() as source:
#         print('Mendengarkan......')
#         suara = mendengar.listen(source, phrase_time_limit=5)
#         try:
#             print('Diterima.....')
#             dengar = mendengar.recognize_google(suara, language='id-ID')
#             print(dengar)
#         except:
#             pass
#         return dengar

whisper_model = whisper.load_model("small")

def perintah():
    duration = 5
    sample_rate = 16000  

    print("Mendengarkan......")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  
    print("Diterima.....")

    audio_data = np.squeeze(audio_data)  
    dengar = whisper_model.transcribe(audio_data, fp16=False, language="id")
    print(dengar["text"])

    return dengar["text"]

In [ ]:
# --- TTS ---

def ngomong(text):
    voices = engine.getProperty('voices')

    for voice in voices:
        if "MSTTS_V110_idID_Andika" in voice.id:
            engine.setProperty('voice', voice.id)
            break

    # Speak the text
    engine.say(text)
    
    # Wait until speaking is finished
    engine.runAndWait()

# mms = VitsModel.from_pretrained("facebook/mms-tts-ind")
# mms_token = AutoTokenizer.from_pretrained("facebook/mms-tts-ind")

# def ngomong(text):
#     inputs = mms_token(text, return_tensors="pt")

#     with torch.no_grad():
#         output = mms(**inputs).waveform
    
#     return Audio(output.squeeze().cpu().numpy(), rate=16000) # Can change rate to make it faster/slower

In [ ]:
# --- MAIN FUNCTION ---

def run_va():
    # prompt from user
    Layanan = perintah()

    # messages = [{"role": "user", "content": Layanan}]
    # prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # print(outputs[0]["generated_text"])

    # ngomong(Layanan)

    # print(Layanan)

    system_prompt= 'Jawab dalam bahasa indonesia.'

    # messages = [
    #     {"role": "question", "content": Layanan}
    #     ]
    messages = [
    {"role": "system", "content": "Tolong jawab singkat."},
    {"role": "user", "content": Layanan}
]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt") 

    start_time = time.time()
    memory_before = psutil.virtual_memory().used

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512, 
        temperature=0.7,
        top_p=0.9,
        do_sample=True

    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    end_time = time.time()
    memory_after = psutil.virtual_memory().used

    inference_time = end_time - start_time
    memory_used = memory_after - memory_before
    cpu_usage = psutil.cpu_percent(interval=1)
    
    print(response)
    print("INFERENCE TIME: ", inference_time)
    print("MEMORY USAGE: ", memory_used)
    print("CPU USAGE: ", cpu_usage)
    ngomong(response)

In [ ]:
run_va()